In [1]:
from config import *

import gensim

import glob
import os
import re

import numpy as np
import pandas as pd

import json

/user/ms5941/.local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from gensim.models import ldamodel

In [3]:
THEME = 'GDP'

In [4]:
# Load dictionary and corpus
dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (THEME, THEME))

2021-04-25 23:32:55,513 : INFO : loading Dictionary object from /work/ms5941/NLP/Temp/GDP/GDP_less_restricted.dict
2021-04-25 23:32:55,519 : INFO : Dictionary lifecycle event {'fname': '/work/ms5941/NLP/Temp/GDP/GDP_less_restricted.dict', 'datetime': '2021-04-25T23:32:55.518595', 'gensim': '4.0.1', 'python': '3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]', 'platform': 'Linux-4.19.0-16-amd64-x86_64-with-debian-10.9', 'event': 'loaded'}


In [ ]:
# 1. Look for ids of related words to theme in the dictionary. This will be a list of ids for each year.
# 

In [5]:
words = pd.read_csv(TEMP_PATH + '/%s_words.csv' % THEME.lower(), index_col=0)

In [6]:
year_name_list = dict()
year_id_list = dict()
for col in words.columns:
    reqd_words = list(words[col].values) + [THEME.lower()]
    reqd_ids = [k for k,v in dictionary_all.items() if v in reqd_words]
    year_name_list[col] = [dictionary_all[i] for i in reqd_ids]
    year_id_list[col] = reqd_ids

In [7]:
year_name_list

{'1996': ['inflation',
  'figure',
  'gdp',
  'economists',
  'unemployment',
  'gross',
  'cpi'],
 '1997': ['consumption',
  'deficit',
  'inflation',
  'gdp',
  'unemployment',
  'gross',
  'cpi'],
 '1998': ['consumption',
  'deficit',
  'inflation',
  'gdp',
  'employment',
  'unemployment',
  'cpi',
  'eci'],
 '1999': ['deficit',
  'inflation',
  'gdp',
  'payrolls',
  'employment',
  'unemployment',
  'cpi'],
 '2000': ['inflation',
  'gdp',
  'payrolls',
  'revised',
  'economists',
  'employment',
  'cpi',
  'pmi'],
 '2001': ['consumption',
  'inflation',
  'figure',
  'gdp',
  'payrolls',
  'revised',
  'employment',
  'gross',
  'reading',
  'cpi'],
 '2002': ['consumption',
  'inflation',
  'gdp',
  'payrolls',
  'employment',
  'gross',
  'reading',
  'cpi'],
 '2003': ['inflation', 'gdp', 'payrolls', 'jobless'],
 '2004': ['inflation', 'output', 'gdp', 'payrolls', 'jobs', 'cpi', 'pmi'],
 '2005': ['inflation',
  'output',
  'gdp',
  'payrolls',
  'jobless',
  'factory',
  'cpi']

In [9]:
years = [str(year) for year in range(START_YEAR, END_YEAR + 1)]

In [10]:
good_words_count_per_year = dict()
for year in years:
    good_words_count_per_year[year] = []
    with open(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year)) as f:
        a = json.load(f)
        for article in a:
            good_words_count_per_year[year].append(len([word for word in article if word in year_name_list[year]]))
    print(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year), 'done!')

/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_1996.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_1997.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_1998.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_1999.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2000.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2001.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2002.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2003.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2004.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2005.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2006.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2007.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2008.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2009.json done!
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2010.json done!
/work/ms5941/NLP/Temp/GDP

In [12]:
theme = 'GDP'

yearly_slices = []
for year in years:
    # Get the 1% threshold count (the top 1% of most relevant articles)
    x = good_words_count_per_year[year]
    x.sort()
    threshold = x[-int(len(x)*0.01):][0]
    year_article_count = 0
    with open(TOKENIZED_ARTICLES_PATH % (theme, theme, year)) as f:
        article_index = 0
        all_articles = json.load(f)
        for articles in all_articles:
            if good_words_count_per_year[year][article_index] >= threshold:
                year_article_count += 1    
            article_index += 1
    yearly_slices.append(year_article_count)
    print(TOKENIZED_ARTICLES_PATH % (theme, theme, year), 'done!')
    print('Count for the year:', year_article_count, 'Vs original:', len(all_articles))

# dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (theme, theme))

/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_1996.json done!
Count for the year: 313 Vs original: 30085
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_1997.json done!
Count for the year: 235 Vs original: 21451
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_1998.json done!
Count for the year: 1403 Vs original: 121502
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_1999.json done!
Count for the year: 890 Vs original: 67258
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2000.json done!
Count for the year: 602 Vs original: 53084
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2001.json done!
Count for the year: 691 Vs original: 57614
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2002.json done!
Count for the year: 555 Vs original: 54160
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2003.json done!
Count for the year: 1028 Vs original: 102461
/work/ms5941/NLP/Temp/GDP/GDP_Articles_Tokenized_2004.json done!
Count for the year: 1121 Vs original: 91690
/work/ms5941/NLP/Temp/G

In [13]:
sum(yearly_slices)

36936

In [20]:
with open(TEMP_PATH + '/%s/%s_yearly_slices.txt' % (THEME, THEME), 'w') as f:
    json.dump(yearly_slices, f)

In [ ]:
# with open(TEMP_PATH + '/%s/%s_yearly_slices.txt' % (THEME, THEME), 'r') as f:
#     yearly_slices = json.load(f)

In [19]:
print(yearly_slices)

[313, 235, 1403, 890, 602, 691, 555, 1028, 1121, 1071, 1720, 2276, 2812, 2574, 1273, 1296, 1032, 1264, 1737, 1722, 2401, 1921, 2852, 3524, 623]


In [21]:
dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (THEME, THEME))
corpus_all = gensim.corpora.MmCorpus(TEMP_PATH + '/%s/%s_less_restricted.mm' % (THEME, THEME))


2021-04-26 00:07:06,671 : INFO : loading Dictionary object from /work/ms5941/NLP/Temp/GDP/GDP_less_restricted.dict
2021-04-26 00:07:06,676 : INFO : Dictionary lifecycle event {'fname': '/work/ms5941/NLP/Temp/GDP/GDP_less_restricted.dict', 'datetime': '2021-04-26T00:07:06.676415', 'gensim': '4.0.1', 'python': '3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]', 'platform': 'Linux-4.19.0-16-amd64-x86_64-with-debian-10.9', 'event': 'loaded'}
2021-04-26 00:07:06,681 : INFO : loaded corpus index from /work/ms5941/NLP/Temp/GDP/GDP_less_restricted.mm.index
2021-04-26 00:07:06,682 : INFO : initializing cython corpus reader from /work/ms5941/NLP/Temp/GDP/GDP_less_restricted.mm
2021-04-26 00:07:06,684 : INFO : accepted corpus with 36936 documents, 1662 features, 4916481 non-zero entries


In [24]:
import numpy as np

cum_yearly_slices = np.cumsum(yearly_slices)

corpus_by_year = dict()
corpus_by_year[START_YEAR] = corpus_all[:cum_yearly_slices[0]]
for i in range(1, 25):
    corpus_by_year[START_YEAR + i] = corpus_all[cum_yearly_slices[i-1]:cum_yearly_slices[i]]

In [26]:
len(corpus_by_year[2009]) 

2574

In [31]:
def generate_lda_model(corpus, dictionary, num_topics=10, passes=50, 
                       iterations=400, eval_every=None, update_every=0, 
                       alpha='auto', eta='auto'):

    lda = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, alpha='auto', eta='auto',
                                 iterations=iterations, num_topics=num_topics, passes=passes, 
                                 eval_every=eval_every, update_every = update_every)
    
    # Save lda model
    tempfile = TEMP_PATH + '/%s/%s_LDA_model_' + '_'.join([str(num_topics), str(passes), str(iterations), str(alpha), str(eta)]) % (THEME, THEME)
    lda.save(temp_file)
    
    return lda



def get_topics(corpus_all, dictionary_all, corpus_by_year, num_topics=10, passes=50, iterations=400, 
               eval_every=None, update_every=0, alpha='auto', eta='auto'):
    """
    Get the top topics for each year, based on an LDA model created using documents of both years
    
    """
    # Check if a model with the same config already exists. 
    # If it does, load the model instead of generating a new one
    tempfile = TEMP_PATH + '/%s/%s_LDA_model_' % (THEME, THEME) + '_'.join([str(num_topics), str(passes), str(iterations), str(alpha), str(eta)]) 
    if os.path.exists(tempfile):
        lda = gensim.models.LdaModel.load(tempfile)
    else:
        lda = generate_lda_model(corpus_all, dictionary_all, num_topics, passes, 
                                 iterations, eval_every, update_every, alpha, eta)

    avg_topics_all = []
    for year in range(START_YEAR, END_YEAR + 1):
        avg_topic_probabilities = get_avg_topic_probabilities(lda, corpus_by_year[year], num_topics)
        df, top_topic_words = get_top_ten_topics_for_year(year, lda, avg_topic_probabilities)
        display(df)
        avg_topics_all.append(avg_topic_probabilities)

    # Plot avg topic distribution graphs
    fig, ax = plt.subplots(1, 2, figsize=(15,5))

    ax[0].bar(range(len(avg_topics_all[0])), avg_topics_all[0]*100)
    ax[1].bar(range(len(avg_topics_all[1])), avg_topics_all[1]*100)
    plt.setp(ax, ylim=(0, 20))
    plt.show()

    return avg_topics_all

In [ ]:
avg_topics_all = get_topics(corpus_all, dictionary_all, corpus_by_year)

2021-04-26 00:10:44,754 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2021-04-26 00:10:44,756 : INFO : using serial LDA version on this node
2021-04-26 00:10:44,758 : INFO : running batch LDA training, 10 topics, 50 passes over the supplied corpus of 36936 documents, updating model once every 36936 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2021-04-26 00:10:44,998 : INFO : PROGRESS: pass 0, at document #2000/36936
2021-04-26 00:11:07,971 : INFO : optimized alpha [0.06315659, 0.078261, 0.060452066, 0.07007666, 0.062615916, 0.059998147, 0.067888364, 0.062345702, 0.062065523, 0.05907267]
2021-04-26 00:11:08,359 : INFO : PROGRESS: pass 0, at document #4000/36936
2021-04-26 00:11:34,530 : INFO : optimized alpha [0.0439105, 0.0660411, 0.04510451, 0.053608447, 0.048513073, 0.046599515, 0.052789237, 0.048386104, 0.047577847, 0.0476005]
2021-04-26 00:11:34,934 : INFO : PROGRESS: p